In [ ]:
#
# Read in files from logs and results/ json/
# RAGAS Pipeline Evaluation
#


In [100]:
! pip install -q pandas
! pip install -q ragas
! pip install -q openpyxl


[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:
import sys
import os
import pandas as pd
import json
import ragas
from datasets import Dataset



In [3]:
log_path =  r'../logs/info.log'

data = []

with open(log_path, 'r') as file:
    for line in file:
        entry = json.loads(line)
        data.append(entry)

info_log_df = pd.DataFrame(data)


In [4]:
info_log_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   timestamp              60 non-null     object
 1   level                  60 non-null     object
 2   function_name          60 non-null     object
 3   original_query         60 non-null     object
 4   generated_query_list   10 non-null     object
 5   current_query          26 non-null     object
 6   semantic_query_result  26 non-null     object
 7   cypher_query           8 non-null      object
 8   cypher_steps           8 non-null      object
 9   context_string         16 non-null     object
 10  generatedResponse      8 non-null      object
dtypes: object(11)
memory usage: 5.3+ KB


In [5]:
info_log_df.head()

,timestamp,level,function_name,original_query,generated_query_list,current_query,semantic_query_result,cypher_query,cypher_steps,context_string,generatedResponse
0,2024-04-21_12-12-46,INFO,generate_multi_question_institution,AACSB Standards on sustainability,[What are the AACSB standards on sustainabilit...,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-04-21_12-12-46,INFO,retriever() - MULTI,AACSB Standards on sustainability,NaN,What are the AACSB standards on sustainability?,[attain participating supporting status depth ...,NaN,NaN,NaN,NaN
2,2024-04-21_12-12-47,INFO,retriever() - MULTI,AACSB Standards on sustainability,NaN,Can you provide information on the AACSB's sus...,[attain participating supporting status depth ...,NaN,NaN,NaN,NaN
3,2024-04-21_12-12-47,INFO,retriever() - MULTI,AACSB Standards on sustainability,NaN,What is the criteria for AACSB accreditation i...,[3.1 faculty sufficiency school adopts applies...,NaN,NaN,NaN,NaN
4,2024-04-21_12-12-50,INFO,retriever() - MULTI CYPHER,AACSB Standards on sustainability,NaN,NaN,NaN,I don't know the answer.,"[{'query': 'MATCH (s:Standard {title: ""AACSB S...",NaN,NaN


In [ ]:
test_query = [
    "AACSB Standards on sustainability", #aacsb standards
    "Our accounting department updated curriculum to include environmental impact in business risk evaluation, does this reflect the sustanability standard", #hybrid subquestion()
    "Standard 5 specifies a systematic process for assurance of learning. What do peer review teams usually expect in determining whether this standard is met?",
    "What are intellectual contributions",
    "Must faculty members publish in order to be qualified as scholarly academic, practice academic, or scholarly practitioner?", #FAQ
    "Who is the dean of my school",
    "What are the management courses",
    "Which leaning objectives are assessed in the undergradute business program ",
    "How is standard 8 defined?",
    "What are MACC program students expected to demonstrate"
]

In [7]:
retriever_df = info_log_df[info_log_df['function_name'] == 'retriever()']


In [16]:
retriever_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8 entries, 5 to 54
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   timestamp              8 non-null      object
 1   level                  8 non-null      object
 2   function_name          8 non-null      object
 3   original_query         8 non-null      object
 4   generated_query_list   0 non-null      object
 5   current_query          0 non-null      object
 6   semantic_query_result  0 non-null      object
 7   cypher_query           0 non-null      object
 8   cypher_steps           0 non-null      object
 9   context_string         8 non-null      object
 10  generatedResponse      0 non-null      object
dtypes: object(11)
memory usage: 768.0+ bytes


In [18]:
extracted_df = retriever_df[['original_query', 'context_string']]
extracted_df.to_json('testing_set.json', orient='records')


In [11]:
# Ground Truth

ground_truth_list = [
    {"original_query": "AACSB Standards on sustainability", "ground_truth": "Sustainability is mentioned throughout the standards, schools must demonstrate the ability to sustain appropriate appropriate resources to achieve their operational and strategic goals for the foreseeable future","qID": 1},
    {"original_query": "Our accounting department updated curriculum to include environmental impact in business risk evaluation, does this reflect the sustanability standard", "ground_truth": "AACSB does not have one particular standard on environmental sustainability, your inclusion of the environmental impact could be seen as a reflection on Standard 9: Engagement and Societal Impact, and/or Standard 4 which relates to curriculum","qID": 2},
    {"original_query": "Standard 5 specifies a systematic process for assurance of learning. What do peer review teams usually expect in determining whether this standard is met?", "ground_truth": "The assurance of learning process is designed to ensure systematic continuous improvement of curriculum. Peer review teams will seek evidence that shows competencies for each degree program are in place. The standard does not describe what a school must do to have a “mature” AoL system. While a robust AoL system will have multiple competency goals for each degree program, AACSB does not prescribe an optimal number of competencies for a given degree program. Typically, schools will close the loop at least once in their accreditation cycle for each competency. AACSB interprets closing the loop to mean that a school shows how curriculum was improved as a result of the assurance of learning process. Results of the assessment should be documented and available for peer review teams upon request. The assessment processes and results should lead to documented continuous improvement in curriculum. Table 5-1 is mandatory for schools in the initial accreditation process and optional for schools completing their CIR report.  However, all schools are encouraged to complete Table 5-1 as it presents assurance of learning data in an organized and standardized way for the benefit of the school and peer review team.","qID": 3},
    {"original_query": "What are intellectual contributions", "ground_truth": "Intellectual contributions are original works intended to advance the theory, practice,  and/or teaching of business. Further, intellectual contributions may have the potential to  address issues of importance to broader society. The contributions are scholarly in the  sense that they are based on generally accepted academic research principles and are disseminated to appropriate audiences. The school’s portfolio of Intellectual contributions may fall into any of the following categories: Basic or Discover Scholarship, Applied or Integrative/Application Scholarship, Teaching and Learning Scholarship","qID": 4},
    {"original_query": "Must faculty members publish in order to be qualified as scholarly academic, practice academic, or scholarly practitioner?", "ground_truth": "Standard 3 provides a non-exhaustive list of activities normally expected of SA, PA, and SP status. While the standard does not specifically require publication of peer- or editorial-reviewed journal articles, schools normally are expected to have guidelines and criteria consistent with their mission and their peers. A peer review team may question a school’s criteria if it appears the criteria are not in alignment with the school’s mission and peer institutions. Note that Standard 8 does require that “A significant cross section of faculty in the school and each discipline is engaged in the production of intellectual contributions, relying heavily on participating faculty.","qID": 5},
    {"original_query": "Who is the dean of my school", "ground_truth": "Dr. Micheal T. Stratton, Dean","qID": 6},
    {"original_query": "What are the management courses", "ground_truth": " MGMT 3101 (Applied Business Statistics), MGMT 3165 (Operations Quality & Supply Chain Management), MGMT 4195 (Strategic Management) , MGMT 4605 (Internship), MGMT 4105 (Human Resource Management) ","qID": 7},
    {"original_query": "Which learning objectives are assessed in the undergraduate business program", "ground_truth": "Ethical Issues, globalization and cross-culturalism, written and oral communication, common business knowledge, effective analysis","qID": 8},
    {"original_query": "How is standard 8 defined?", "ground_truth": "STANDARD 8: IMPACT OF SCHOLARSHIP 8.1 The school’s faculty collectively produce high-quality, impactful intellectual contributions that, over time, develop into mission-consistent areas of thought leadership for the school. 8.2 The school collaborates with a wide variety of external stakeholders to create and transfer credible, relevant, and timely knowledge that informs the theory, policy, and/or practice of business to develop into mission-consistent areas of thought leadership for the school. 8.3 The school’s portfolio of intellectual contributions contains exemplars of basic, applied, and/or pedagogical research that have had a positive societal impact, consistent with the school’s mission.","qID": 9},
    {"original_query": "What are MACC program students expected to demonstrate", "ground_truth": "Advanced knowledge and theory of accounting.","qID": 10}
]

In [12]:
ground_truth_df = pd.DataFrame(ground_truth_list)

In [13]:
ground_truth_df

,original_query,ground_truth,qID
0,AACSB Standards on sustainability,Sustainability is mentioned throughout the sta...,1
1,Our accounting department updated curriculum t...,AACSB does not have one particular standard on...,2
2,Standard 5 specifies a systematic process for ...,The assurance of learning process is designed ...,3
3,What are intellectual contributions,Intellectual contributions are original works ...,4
4,Must faculty members publish in order to be qu...,Standard 3 provides a non-exhaustive list of a...,5
5,Who is the dean of my school,"Dr. Micheal T. Stratton, Dean",6
6,What are the management courses,"MGMT 3101 (Applied Business Statistics), MGMT...",7
7,Which learning objectives are assessed in the ...,"Ethical Issues, globalization and cross-cultur...",8
8,How is standard 8 defined?,STANDARD 8: IMPACT OF SCHOLARSHIP 8.1 The scho...,9
9,What are MACC program students expected to dem...,Advanced knowledge and theory of accounting.,10


In [19]:
context_str_df= pd.read_json('testing_set.json')

In [20]:
context_str_df

,qID,original_query,context_string
0,1,AACSB Standards on sustainability,attain participating supporting status depth b...
1,2,Our accounting department updated curriculum t...,3.1 faculty sufficiency school adopts applies ...
2,3,Standard 5 specifies a systematic process for ...,programs student learning outcomes evidence le...
3,4,What are intellectual contributions,behavior . addition categorization intellectua...
4,5,Must faculty members publish in order to be qu...,. faculty qualifications status refers one fou...
5,7,What are the management courses,accreditation process complete table 5 1 degre...
6,8,Which learning objectives are assessed in the ...,The J. Whitney Bunting College of Business 201...
7,9,How is standard 8 defined?,standard 8 impact scholarship 8.1 school facul...


In [23]:
merged_df = ground_truth_df.merge(context_str_df, on='qID', how = 'left')

In [27]:
merged_df.to_csv('testing_set.csv', index=False)

In [78]:
to_json_df = pd.read_csv('../eval_data_set/tsf_2.csv')
to_json_df.to_json('../eval_data_set/testing_set_formatted.json', orient = 'records')

In [79]:
json_path = '../eval_data_set/testing_set_formatted.json'

with open(json_path, 'r') as json_file:
    json_data = json.load(json_file)

In [80]:
#json_data

[{'qID': 1,
  'question': 'AACSB Standards on sustainability',
  'ground_truths': 'Sustainability is mentioned throughout the standards, schools must demonstrate the ability to sustain appropriate appropriate resources to achieve their operational and strategic goals for the foreseeable future',
  'answer': "The AACSB (Association to Advance Collegiate Schools of Business) has established standards on sustainability that schools must adhere to in order to attain and maintain participating supporting status. These standards are periodically reviewed to ensure they reflect a focus on continuous improvement.\r\n\r\nOne of the criteria for attaining participating supporting status is the sufficiency of faculty members. The school must have faculty members who are actively participating and supporting the mission of the school. The criteria for faculty sufficiency should be documented and consistent with the mission of the school. The school may adapt guidance based on their specific situat

In [81]:
import ast
# dataset_dict = {"qID": [], "question": [],"ground_truth":[],"answer": [], "contexts": []}
# for item in json_data:
#     dataset_dict["qID"].append(item["qID"])
#     dataset_dict["question"].append(item["question"])
#     dataset_dict["ground_truth"].append(item["ground_truths"])
#     dataset_dict["answer"].append(item["answer"])
#     dataset_dict["contexts"].append(item["contexts"])

dataset_dict = {"qID": [], "question": [], "ground_truth": [], "answer": [], "contexts": []}
for item in json_data:
    try:
        dataset_dict["qID"].append(item["qID"])
        dataset_dict["question"].append(item["question"])
        dataset_dict["ground_truth"].append(item["ground_truths"])
        dataset_dict["answer"].append(item["answer"])
        dataset_dict["contexts"].append(ast.literal_eval(item["contexts"])) #convert string to list
    except json.JSONDecodeError as e:
        print(f"Error processing line: {line}")
        print(f"JSONDecodeError: {e}")

In [71]:
print(type(dataset_dict["contexts"][0]))

<class 'str'>


### RAGAS EVALUATION

In [82]:
#
# Source: https://docs.ragas.io/en/latest/getstarted/evaluation.html#get-started-evaluation
#



dataset = Dataset.from_dict(dataset_dict)

In [126]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_relevancy,
    answer_correctness
)

In [127]:
from ragas import evaluate

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAIKEY")

result = evaluate(
    dataset,
    metrics=[
        context_relevancy,
        faithfulness,
        answer_relevancy,
        context_recall,
        answer_correctness
    ],
    raise_exceptions= False
)

result

Evaluating:   0%|          | 0/45 [00:00<?, ?it/s]No statements were generated from the answer.
No statements were generated from the answer.
Evaluating:  98%|█████████▊| 44/45 [00:31<00:03,  3.40s/it]Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\Candace Edwards\capstone\advRAG\myenv\lib\site-packages\ragas\executor.py", line 79, in _aresults
    r = await future
  File "C:\Users\Candace Edwards\AppData\Local\Programs\Python\Python310\lib\asyncio\tasks.py", line 571, in _wait_for_one
    return f.result()  # May raise f.exception().
  File "c:\Users\Candace Edwards\capstone\advRAG\myenv\lib\site-packages\ragas\executor.py", line 38, in sema_coro
    return await coro
  File "c:\Users\Candace Edwards\capstone\advRAG\myenv\lib\site-packages\ragas\executor.py", line 112, in wrapped_callable_async
    return counter, await callable(*args, **kwargs)
  File "c:\Users\Candace Edwards\capstone\advRAG\myenv\lib\site-packages\ragas\metrics\base.py", l

{'context_relevancy': 0.4945, 'faithfulness': 0.2515, 'answer_relevancy': 1.0000, 'context_recall': 0.7970, 'answer_correctness': 0.7870}

In [112]:
#score = evaluate(dataset,metrics=[answer_correctness])

In [128]:
df = result.to_pandas()
df.fillna(0,inplace=True)
df

,qID,question,ground_truth,answer,contexts,context_relevancy,faithfulness,answer_relevancy,context_recall,answer_correctness
0,1,AACSB Standards on sustainability,Sustainability is mentioned throughout the sta...,The AACSB (Association to Advance Collegiate S...,[attain participating supporting status depth ...,0.000000,0.000000,1.0,0.780485,1.000000
1,2,Our accounting department updated curriculum t...,AACSB does not have one particular standard on...,"Based on the information provided, it appears ...",[3.1 faculty sufficiency school adopts applies...,0.216925,0.000000,1.0,0.915368,1.000000
2,3,Standard 5 specifies a systematic process for ...,The assurance of learning process is designed ...,Peer review teams usually expect several key e...,[programs student learning outcomes evidence l...,0.873494,0.142857,1.0,0.937497,1.000000
3,4,What are intellectual contributions,Intellectual contributions are original works ...,Intellectual contributions refer to the variou...,[behavior . addition categorization intellectu...,0.793183,1.000000,1.0,0.983644,0.750000
4,5,Must faculty members publish in order to be qu...,Standard 3 provides a non-exhaustive list of a...,In order to be qualified as a scholarly academ...,[. faculty qualifications status refers one fo...,0.429108,0.944444,1.0,0.899504,0.500000
5,6,Who is the dean of my school,"Dr. Micheal T. Stratton, Dean",none,[],0.470764,0.000000,0.0,0.000000,0.000000
6,7,What are the management courses,"MGMT 3101 (Applied Business Statistics), MGMT...",Management courses are a part of the degree pr...,[accreditation process complete table 5 1 degr...,0.194339,0.000000,1.0,0.900843,0.833333
7,9,How is standard 8 defined?,STANDARD 8: IMPACT OF SCHOLARSHIP 8.1 The scho...,Standard 8 is defined as the impact scholarshi...,[standard 8 impact scholarship 8.1 school facu...,0.200759,0.176471,1.0,0.959057,1.000000
8,10,What are MACC program students expected to dem...,Advanced knowledge and theory of accounting.,0,[],0.777441,0.000000,0.0,0.000000,1.000000


In [129]:
import openpyxl


output_path = r"..\results\RAGAS_pipeline_evaluation\full_dataframe.xlsx"
df.to_excel(output_path, index=False) 

In [130]:
summary_stats = {
    'Metric': ['Mean', 'Median', 'Min', 'Max'],
    'Context Relevancy': [df['context_relevancy'].mean(), df['context_relevancy'].median(), df['context_relevancy'].min(), df['context_relevancy'].max()],
    'Faithfulness': [df['faithfulness'].mean(), df['faithfulness'].median(), df['faithfulness'].min(), df['faithfulness'].max()],
    'Answer Relevancy': [df['answer_relevancy'].mean(), df['answer_relevancy'].median(), df['answer_relevancy'].min(), df['answer_relevancy'].max()],
    'Context Recall': [df['context_recall'].mean(), df['context_recall'].median(), df['context_recall'].min(), df['context_recall'].max()],
    'Answer Correctness': [df['answer_correctness'].mean(), df['answer_correctness'].median(), df['answer_correctness'].min(), df['answer_correctness'].max()],
}

In [131]:
summary_df = pd.DataFrame(summary_stats)
summary_df

,Metric,Context Relevancy,Faithfulness,Answer Relevancy,Context Recall,Answer Correctness
0,Mean,0.439557,0.25153,0.777778,0.708489,0.787037
1,Median,0.429108,0.00000,1.000000,0.900843,1.000000
2,Min,0.000000,0.00000,0.000000,0.000000,0.000000
3,Max,0.873494,1.00000,1.000000,0.983644,1.000000


In [132]:
output_path = r"..\results\RAGAS_pipeline_evaluation\results.xlsx"
summary_df.to_excel(output_path, index=False)  

